# 背景與動機 🎯

1. 農業現代化的需求：現代農業需要科技的幫助，以提高效率、節省成本。決策系統可以為農民提供關鍵的決策支持，以提高農業效率。

2. 土壤管理和作物選擇：不同的作物對土壤的需求各不相同，而土壤的品質和成分也因地點而異。農作物決策系統可以挑選適合當地土壤和氣候條件的作物，實現更高的農業生產力。

我們希望提供農業社區一個先進的工具，幫助農民在開始種植作物之前做出明智的決定，來提高生產力、節省資源並實現可持續的農業實踐。幫助農民在種植作物方面做出更明智的選擇，提高農業生產力，降低風險，促進農業的永續發展。

# 資料集來源

由農業開放資料平臺取得：
1. 雲嘉南地區土壤性質分析資料 [Data detail](https://data.coa.gov.tw/open_detail.aspx?id=116)
2. 臺東區植體及土壤診斷分析服務資料 [Data detail](https://data.coa.gov.tw/open_detail.aspx?id=142)

# 預期結果

農作物決策系統預期將幫助農民在種植作物方面做出更明智的選擇，提高農業生產力，降低風險，促進農業的永續發展。

# 研究方法與步驟

### 資料處理

In [82]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')

In [83]:
df1 = pd.read_csv('/Users/jiangjiaen/Desktop/DSS/COA_OpenData.csv')  # 雲嘉南
df2 = pd.read_csv('/Users/jiangjiaen/Desktop/DSS/142.csv')  # 台東

In [84]:
df1 = df1.drop(columns=['送樣日期', '樣品地段地號', '鈉Na'])
df2 = df2.drop(columns=['年度', '月份', '樣品編號', '地點', 'Fe', 'Mn', 'Cu', 'Zn', 'Pb', 'Cd', 'Ni', 'Cr', 'Si', 'Na'])


In [85]:
# 對df1進行欄位重新命名
df1 = df1.rename(columns={
    '種植作物': 'CropType',
    '有機質': 'OrganicMatter',
    '有效性磷': 'P2O5',
    '有效性鉀': 'K2O',
    '有效性鈣': 'CaO',
    '有效性鎂': 'MgO',
})

# 對df2進行欄位重新命名
df2 = df2.rename(columns={
    '試驗項目': 'CropType',
    '有機質': 'OrganicMatter'
})

# 進行合併或其他操作
df = pd.concat([df1, df2], ignore_index=True)

In [86]:
# 創建一個字典，將df1、df2品種相同，但命名不同的水果名統一
replace_dict = {
    '紅龍果': '火龍果',
    '茶葉': '茶',
    '米': '水稻',
    '硬質玉米': '玉米',
    '小番茄': '番茄',
    '文旦': '文旦柚',
    '鳳梨釋迦': '釋迦',
    '胡蘿蔔': '紅蘿蔔'
}

# 使用replace方法進行替換
df['CropType'] = df['CropType'].replace(replace_dict)


In [87]:
df = df.dropna(axis=0)
df.isnull().sum()

CropType         0
EC               0
pH               0
OrganicMatter    0
P2O5             0
K2O              0
CaO              0
MgO              0
dtype: int64

In [88]:
cols_to_convert = df.columns[df.columns != 'CropType']

for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [89]:
# 創建布林遮罩，用於識別那些計數大於或等於190的種植作物
counts = df['CropType'].value_counts()
mask = df['CropType'].isin(counts[counts >= 190].index)
mask

0        False
1        False
2        False
3        False
4        False
         ...  
15773    False
15774    False
15775    False
15776    False
15777     True
Name: CropType, Length: 14546, dtype: bool

In [90]:
df = df[mask]

In [91]:
df

,CropType,EC,pH,OrganicMatter,P2O5,K2O,CaO,MgO
10,火龍果,0.446,6.12,2.203448,1473.235647,359.372812,3941.156951,264.158337
12,文旦柚,0.112,6.49,1.956897,15.683317,75.804170,1860.018160,318.911364
13,文旦柚,0.114,7.00,1.386207,12.817032,52.386375,1816.381067,299.187349
14,文旦柚,0.214,7.75,0.946552,49.547788,63.205538,1843.764998,142.196856
15,文旦柚,0.196,8.44,1.139655,116.654290,70.876705,2581.730594,364.377548
...,...,...,...,...,...,...,...,...
15769,水稻,0.230,6.38,4.320000,58.770000,260.360000,5874.230000,659.330000
15770,水稻,0.310,6.34,2.460000,53.680000,112.160000,9110.360000,103.230000
15771,水稻,0.110,5.86,5.080000,163.160000,1073.670000,3371.260000,377.770000
15772,玉米,0.160,5.96,2.710000,42.510000,164.520000,3036.610000,871.530000


### 分離特徵和目標標籤

In [92]:
features = df[['EC', 'pH', 'OrganicMatter', 'P2O5', 'K2O', 'CaO', 'MgO']]
target = df['CropType']
labels = df['CropType']

In [93]:
# 初始化空列表
acc = []
model = []

In [94]:
# 以平均數填充
features_filled = features.fillna(features.mean())

In [95]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features_filled, target, test_size=0.2, random_state=0)

### Random Forest

In [96]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=1300, random_state=0)
RF.fit(Xtrain,Ytrain)

predicted_values = RF.predict(Xtest)

x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('RandomForest')
print("RandomForest's Accuracy is: ", x)

print(classification_report(Ytest,predicted_values))

RandomForest's Accuracy is:  0.6593059936908517
              precision    recall  f1-score   support

          小米       0.89      0.69      0.78        48
         文旦柚       0.63      0.86      0.73       374
          水稻       0.74      0.81      0.77       251
         火龍果       0.71      0.16      0.26        95
          玉米       0.56      0.47      0.51        64
          番茄       0.58      0.49      0.53        92
         紅蘿蔔       0.69      0.62      0.65        65
          芒果       0.59      0.52      0.55        50
          花生       0.65      0.55      0.60        62
          蔬菜       0.80      0.19      0.31        63
          釋迦       0.63      0.71      0.67       104

    accuracy                           0.66      1268
   macro avg       0.68      0.55      0.58      1268
weighted avg       0.67      0.66      0.63      1268



### 使用者介面設計

In [97]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

def predict_crop():
    # 檢查是否有空白輸入
    if any(entry.get() == "" for entry in [EC_entry, pH_entry, OrganicMatter_entry, P2O5_entry, K2O_entry, CaO_entry, MgO_entry]):
        messagebox.showerror("Error", "Please fill in all input labels.", icon='warning')
        return
    
    try:
        data = [
            float(EC_entry.get()),
            float(pH_entry.get()),
            float(OrganicMatter_entry.get()),
            float(P2O5_entry.get()),
            float(K2O_entry.get()),
            float(CaO_entry.get()),
            float(MgO_entry.get())
        ]
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numeric values.", icon='error')
        return
    
    predicted_crop = RF.predict([data])[0]
    
    result_label.config(text=f"Recommended CropType: {predicted_crop}")

# window
root = tk.Tk()
root.geometry("800x600")
root.title("Crop Prediction")
font_size = 16

# entry label
tk.Label(root, text="EC", anchor="e", padx=150, font=("Arial", font_size)).grid(row=0, column=0, pady=15)
EC_entry = tk.Entry(root, justify='center', font=("Arial", font_size))
EC_entry.grid(row=0, column=1, pady=15)

tk.Label(root, text="pH", anchor="e", padx=150, font=("Arial", font_size)).grid(row=1, column=0, pady=15)
pH_entry = tk.Entry(root, justify='center', font=("Arial", font_size))
pH_entry.grid(row=1, column=1, pady=15)

tk.Label(root, text="OrganicMatter", anchor="e", padx=150, font=("Arial", font_size)).grid(row=2, column=0, pady=15)
OrganicMatter_entry = tk.Entry(root, justify='center', font=("Arial", font_size))
OrganicMatter_entry.grid(row=2, column=1, pady=15)

tk.Label(root, text="P2O5", anchor="e", padx=150, font=("Arial", font_size)).grid(row=3, column=0, pady=15)
P2O5_entry = tk.Entry(root, justify='center', font=("Arial", font_size))
P2O5_entry.grid(row=3, column=1, pady=15)

tk.Label(root, text="K2O", anchor="e", padx=150, font=("Arial", font_size)).grid(row=4, column=0, pady=15)
K2O_entry = tk.Entry(root, justify='center', font=("Arial", font_size))
K2O_entry.grid(row=4, column=1, pady=15)

tk.Label(root, text="CaO", anchor="e", padx=150, font=("Arial", font_size)).grid(row=5, column=0, pady=15)
CaO_entry = tk.Entry(root, justify='center', font=("Arial", font_size))
CaO_entry.grid(row=5, column=1, pady=15)

tk.Label(root, text="MgO", anchor="e", padx=150, font=("Arial", font_size)).grid(row=6, column=0, pady=15)
MgO_entry = tk.Entry(root, justify='center', font=("Arial", font_size))
MgO_entry.grid(row=6, column=1, pady=15)

# button
style = ttk.Style()
style.configure('TButton', justify='center', font=('Arial', font_size+2), padding=(10, 5))
predict_button = ttk.Button(root, text="Predict Crop", command=predict_crop, style='TButton')
predict_button.grid(row=7, column=0, columnspan=2, pady=(30,20))

# result label
result_label = ttk.Label(root, text="Recommended CropType: ", font=("Arial", font_size+4, "bold"))
result_label.grid(row=8, column=0, columnspan=2, pady=(20, 40))

# result_label = ttk.Label(root, text="Recommended CropType: ", font=("Arial", font_size+2, "bold"))
# result_label.grid(row=8, column=0, columnspan=2, pady=(20,40))

root.mainloop()